In [37]:
# spark 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession, Row

# python op 
import pandas as pd
import io
import requests

In [17]:
# config 
conf = SparkConf().setAppName("load UBER data")
sc = SparkContext(conf=conf)
sqlCtx = SQLContext(sc)
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [6]:
#url='https://github.com/yennanliu/uber-tlc-foil-response/blob/master/uber-trip-data/uber-raw-data-jun14.csv'
#s=requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('utf-8')))


In [7]:
cd

/Users/yennanliu


In [8]:
cd uber-tlc-foil-response/

/Users/yennanliu/uber-tlc-foil-response


In [9]:
ls 

Aggregate FHV Data.xlsx* TLC_letter2.pdf          other-FHV-data/
README.md                TLC_letter3.pdf          uber-trip-data/
TLC_letter.pdf           Uber-Jan-Feb-FOIL.csv*


In [18]:
# access DATA here https://github.com/yennanliu/uber-tlc-foil-response
data = sc.textFile("Uber-Jan-Feb-FOIL.csv")

In [24]:
df=pd.read_csv("Uber-Jan-Feb-FOIL.csv")

In [25]:
df.columns

Index(['dispatching_base_number', 'date', 'active_vehicles', 'trips'], dtype='object')

In [21]:
data2 = sc.textFile("Uber-Jan-Feb-FOIL.csv").map(lambda line: line.split(","))



In [20]:
data.take(4)

['dispatching_base_number,date,active_vehicles,trips',
 'B02512,1/1/2015,190,1132',
 'B02765,1/1/2015,225,1765',
 'B02764,1/1/2015,3427,29421']

In [26]:
data2 = sc.textFile("Uber-Jan-Feb-FOIL.csv").map(lambda line: line.split(","))
headers = data2.first()
traindata = data2.filter(lambda row: row != headers)
sqlContext = SQLContext(sc)
dataFrame = sqlContext.createDataFrame(traindata, ['dispatching_base_number', 'date', 'active_vehicles', 'trips'])





In [27]:
dataFrame.show()

+-----------------------+--------+---------------+-----+
|dispatching_base_number|    date|active_vehicles|trips|
+-----------------------+--------+---------------+-----+
|                 B02512|1/1/2015|            190| 1132|
|                 B02765|1/1/2015|            225| 1765|
|                 B02764|1/1/2015|           3427|29421|
|                 B02682|1/1/2015|            945| 7679|
|                 B02617|1/1/2015|           1228| 9537|
|                 B02598|1/1/2015|            870| 6903|
|                 B02598|1/2/2015|            785| 4768|
|                 B02617|1/2/2015|           1137| 7065|
|                 B02512|1/2/2015|            175|  875|
|                 B02682|1/2/2015|            890| 5506|
|                 B02765|1/2/2015|            196| 1001|
|                 B02764|1/2/2015|           3147|19974|
|                 B02765|1/3/2015|            201| 1526|
|                 B02617|1/3/2015|           1188|10664|
|                 B02598|1/3/20

In [29]:
ls uber-trip-data

taxi-zone-lookup.csv              uber-raw-data-jul14.csv
uber-raw-data-apr14.csv           uber-raw-data-jun14.csv
uber-raw-data-aug14.csv           uber-raw-data-may14.csv
uber-raw-data-janjune-15.csv.zip  uber-raw-data-sep14.csv


In [34]:
data3 = sc.textFile("uber-trip-data/uber-raw-data-apr14.csv").map(lambda line: line.split(","))


In [35]:
data3.take(3)

[['"Date/Time"', '"Lat"', '"Lon"', '"Base"'],
 ['"4/1/2014 0:11:00"', '40.769', '-73.9549', '"B02512"'],
 ['"4/1/2014 0:17:00"', '40.7267', '-74.0345', '"B02512"']]

In [75]:
dataFrame = dataFrame.withColumn("date_", dataFrame["date"].cast("timestamp"))
dataFrame = dataFrame.withColumn("trips", dataFrame["trips"].cast("float"))
dataFrame = dataFrame.withColumn("active_vehicles", dataFrame["active_vehicles"].cast("float"))
dataFrame.show()

+-----------------------+--------+---------------+-------+-----+
|dispatching_base_number|    date|active_vehicles|  trips|date_|
+-----------------------+--------+---------------+-------+-----+
|                 B02512|1/1/2015|          190.0| 1132.0| null|
|                 B02765|1/1/2015|          225.0| 1765.0| null|
|                 B02764|1/1/2015|         3427.0|29421.0| null|
|                 B02682|1/1/2015|          945.0| 7679.0| null|
|                 B02617|1/1/2015|         1228.0| 9537.0| null|
|                 B02598|1/1/2015|          870.0| 6903.0| null|
|                 B02598|1/2/2015|          785.0| 4768.0| null|
|                 B02617|1/2/2015|         1137.0| 7065.0| null|
|                 B02512|1/2/2015|          175.0|  875.0| null|
|                 B02682|1/2/2015|          890.0| 5506.0| null|
|                 B02765|1/2/2015|          196.0| 1001.0| null|
|                 B02764|1/2/2015|         3147.0|19974.0| null|
|                 B02765|

In [76]:
dataFrame.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: float (nullable = true)
 |-- trips: float (nullable = true)
 |-- date_: timestamp (nullable = true)



In [53]:
from pyspark.sql.functions import unix_timestamp

(sc
    .parallelize([Row(dt='2016_08_21 11_31_08')])
    .toDF()
    .withColumn("parsed", unix_timestamp("dt", "yyyy_MM_dd")
    .cast("double")
    .cast("timestamp"))
    .show(1, False))

+-------------------+-------------------+
|dt                 |parsed             |
+-------------------+-------------------+
|2016_08_21 11_31_08|2016-08-21 00:00:00|
+-------------------+-------------------+



In [49]:
dataFrame.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: string (nullable = true)
 |-- trips: string (nullable = true)



In [78]:
dataFrame.groupby('dispatching_base_number').count().show()

+-----------------------+-----+
|dispatching_base_number|count|
+-----------------------+-----+
|                 B02512|   59|
|                 B02598|   59|
|                 B02682|   59|
|                 B02765|   59|
|                 B02617|   59|
|                 B02764|   59|
+-----------------------+-----+



In [91]:
spark_RDD.take(3)

[Row(dispatching_base_number='B02512', date='1/1/2015', active_vehicles=190.0, trips=1132.0, date_=None),
 Row(dispatching_base_number='B02765', date='1/1/2015', active_vehicles=225.0, trips=1765.0, date_=None),
 Row(dispatching_base_number='B02764', date='1/1/2015', active_vehicles=3427.0, trips=29421.0, date_=None)]

In [111]:
	spark_RDD = dataFrame.rdd 
	top_batch = spark_RDD\
				.map(lambda x : (x.dispatching_base_number , 20))\
				.reduceByKey(add)\
				.sortBy(lambda x : x[1], False)\
				.take(30)

In [112]:
top_batch

[('B02598', 1180),
 ('B02682', 1180),
 ('B02765', 1180),
 ('B02617', 1180),
 ('B02764', 1180),
 ('B02512', 1180)]

In [109]:
from operator import add


spark_RDD\
.map(lambda x : (x.dispatching_base_number , 20))\
.reduceByKey(add)\
.sortBy(lambda x : x[1], False)\
.take(10)


[('B02598', 1180),
 ('B02682', 1180),
 ('B02765', 1180),
 ('B02617', 1180),
 ('B02764', 1180),
 ('B02512', 1180)]

In [88]:
spark_RDD = dataFrame.rdd 
top_batch = spark_RDD\
            .map(lambda x : (x.dispatching_base_number , 1)\
            .reduceByKey(add))\
            .sortBy(lambda x : x[1], False)\
            .take(30)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 122.0 failed 1 times, most recent failure: Lost task 0.0 in stage 122.0 (TID 1521, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yennanliu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/Users/yennanliu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/yennanliu/anaconda3/envs/pyspark_/lib/python3.5/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/yennanliu/anaconda3/envs/pyspark_/lib/python3.5/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File 

In [86]:
dataFrame.agg({'trips':'mean', 'trips':'count'}).show()

+------------+
|count(trips)|
+------------+
|         354|
+------------+



In [79]:
dataFrame.groupby('dispatching_base_number').sum().show()

+-----------------------+--------------------+----------+
|dispatching_base_number|sum(active_vehicles)|sum(trips)|
+-----------------------+--------------------+----------+
|                 B02512|             13125.0|   93786.0|
|                 B02598|             58653.0|  540791.0|
|                 B02682|             71431.0|  662509.0|
|                 B02765|             22575.0|  193670.0|
|                 B02617|             79758.0|  725025.0|
|                 B02764|            217290.0| 1914449.0|
+-----------------------+--------------------+----------+



In [77]:
dataFrame.groupby('trips').mean().show()

+-------+--------------------+----------+
|  trips|avg(active_vehicles)|avg(trips)|
+-------+--------------------+----------+
| 1001.0|               196.0|    1001.0|
| 1582.0|               233.0|    1582.0|
| 1521.0|               177.0|    1521.0|
| 7565.0|              1084.0|    7565.0|
| 1438.0|               235.0|    1438.0|
| 9600.0|              1072.0|    9600.0|
|33802.0|              3831.0|   33802.0|
|11998.0|              1619.0|   11998.0|
|14975.0|              1510.0|   14975.0|
| 2410.0|               299.0|    2410.0|
| 2086.0|               295.0|    2086.0|
|12977.0|              1445.0|   12977.0|
| 9537.0|              1228.0|    9537.0|
| 1608.0|               241.5|    1608.0|
|12016.0|              1394.0|   12016.0|
|28266.0|              3543.0|   28266.0|
|12665.0|              1350.0|   12665.0|
|11640.0|              1181.0|   11640.0|
| 8397.0|               974.0|    8397.0|
|13462.0|              1463.0|   13462.0|
+-------+--------------------+----

In [67]:
dataFrame.groupby('dispatching_base_number').avg().show()

+-----------------------+
|dispatching_base_number|
+-----------------------+
|                 B02512|
|                 B02598|
|                 B02682|
|                 B02765|
|                 B02617|
|                 B02764|
+-----------------------+



In [69]:
dataFrame.show()

+-----------------------+--------+---------------+-----+-----+
|dispatching_base_number|    date|active_vehicles|trips|date_|
+-----------------------+--------+---------------+-----+-----+
|                 B02512|1/1/2015|            190| 1132| null|
|                 B02765|1/1/2015|            225| 1765| null|
|                 B02764|1/1/2015|           3427|29421| null|
|                 B02682|1/1/2015|            945| 7679| null|
|                 B02617|1/1/2015|           1228| 9537| null|
|                 B02598|1/1/2015|            870| 6903| null|
|                 B02598|1/2/2015|            785| 4768| null|
|                 B02617|1/2/2015|           1137| 7065| null|
|                 B02512|1/2/2015|            175|  875| null|
|                 B02682|1/2/2015|            890| 5506| null|
|                 B02765|1/2/2015|            196| 1001| null|
|                 B02764|1/2/2015|           3147|19974| null|
|                 B02765|1/3/2015|            201| 1526

In [60]:
spark_RDD = dataFrame.rdd
digested_RDD = spark_RDD.map(
              lambda x: Row(
              author_ip = x['trips'],
             timestamp = x['date']))\
             .filter(lambda x : x['timestamp'] >= '2018-01-01')\
              .take(30)

In [61]:
digested_RDD

[Row(author_ip='1132', timestamp='1/1/2015'),
 Row(author_ip='1765', timestamp='1/1/2015'),
 Row(author_ip='29421', timestamp='1/1/2015'),
 Row(author_ip='7679', timestamp='1/1/2015'),
 Row(author_ip='9537', timestamp='1/1/2015'),
 Row(author_ip='6903', timestamp='1/1/2015'),
 Row(author_ip='4768', timestamp='1/2/2015'),
 Row(author_ip='7065', timestamp='1/2/2015'),
 Row(author_ip='875', timestamp='1/2/2015'),
 Row(author_ip='5506', timestamp='1/2/2015'),
 Row(author_ip='1001', timestamp='1/2/2015'),
 Row(author_ip='19974', timestamp='1/2/2015'),
 Row(author_ip='1526', timestamp='1/3/2015'),
 Row(author_ip='10664', timestamp='1/3/2015'),
 Row(author_ip='7432', timestamp='1/3/2015'),
 Row(author_ip='8010', timestamp='1/3/2015'),
 Row(author_ip='1088', timestamp='1/3/2015'),
 Row(author_ip='29729', timestamp='1/3/2015'),
 Row(author_ip='791', timestamp='1/4/2015'),
 Row(author_ip='5621', timestamp='1/4/2015'),
 Row(author_ip='5223', timestamp='1/4/2015'),
 Row(author_ip='993', timestamp=

In [113]:
#########

In [114]:
ls

Aggregate FHV Data.xlsx* TLC_letter3.pdf          other-FHV-data/
README.md                Uber-Jan-Feb-FOIL.csv*   uber-trip-data/
TLC_letter.pdf           derby.log
TLC_letter2.pdf          metastore_db/


In [115]:
ls uber-trip-data/

taxi-zone-lookup.csv              uber-raw-data-jul14.csv
uber-raw-data-apr14.csv           uber-raw-data-jun14.csv
uber-raw-data-aug14.csv           uber-raw-data-may14.csv
uber-raw-data-janjune-15.csv.zip  uber-raw-data-sep14.csv


In [134]:
# load all trip csv 

df_ = spark.read.format("csv").option("header", "true").load("uber-trip-data/uber-raw-data-*.csv")
#spark.read.option("header", "true").csv("../Downloads/*.csv")



In [133]:
df_2 = spark.read.format("csv").option("header", "true").load("uber-trip-data/uber-raw-data-apr14.csv")



In [140]:
spark_RDD = df_2.rdd
date_count = spark_RDD\
            .map(lambda x : (x['Date/Time'], 1))\
            .reduceByKey(add)\
            .take(30)
print (date_count)

[('9/30/2014 12:18:00', 14), ('8/7/2014 18:28:00', 49), ('7/26/2014 17:04:00', 19), ('6/19/2014 4:13:00', 4), ('7/29/2014 6:31:00', 26), ('4/18/2014 1:44:00', 4), ('5/19/2014 5:40:00', 8), ('6/24/2014 22:29:00', 17), ('4/7/2014 2:26:00', 2), ('9/30/2014 11:36:00', 14), ('6/21/2014 1:37:00', 10), ('4/15/2014 2:02:00', 3), ('9/24/2014 14:20:00', 30), ('9/13/2014 22:49:00', 23), ('5/25/2014 8:43:00', 6), ('5/12/2014 15:52:00', 18), ('8/6/2014 5:34:00', 14), ('9/21/2014 9:47:00', 15), ('9/13/2014 18:20:00', 45), ('6/26/2014 18:56:00', 34), ('6/7/2014 5:09:00', 1), ('8/22/2014 9:35:00', 22), ('4/10/2014 18:55:00', 21), ('7/9/2014 11:12:00', 18), ('7/29/2014 4:07:00', 4), ('7/2/2014 2:17:00', 6), ('6/5/2014 13:06:00', 14), ('9/14/2014 22:46:00', 12), ('5/4/2014 15:35:00', 16), ('7/6/2014 13:20:00', 16)]


In [125]:
df_.printSchema()

root
 |-- Date/Time: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Lon: string (nullable = true)
 |-- Base: string (nullable = true)



In [136]:
df_2.count()

564516

In [135]:
df_.count()

4534327

In [139]:
df_.groupby('Date/Time').count().show()

+-----------------+-----+
|        Date/Time|count|
+-----------------+-----+
| 9/1/2014 0:01:00|   18|
| 9/1/2014 0:15:00|   13|
|9/1/2014 14:57:00|   20|
| 9/2/2014 7:48:00|   29|
|9/2/2014 10:40:00|   20|
|9/2/2014 12:26:00|   17|
|9/2/2014 18:00:00|   26|
|9/2/2014 21:54:00|   24|
|9/2/2014 21:59:00|   22|
| 9/3/2014 7:30:00|   28|
| 9/3/2014 8:41:00|   41|
|9/3/2014 11:44:00|   27|
|9/3/2014 17:50:00|   37|
| 9/4/2014 7:27:00|   33|
|9/4/2014 10:27:00|   29|
| 9/5/2014 8:08:00|   29|
|9/5/2014 18:23:00|   61|
| 9/6/2014 0:37:00|   25|
| 9/6/2014 6:48:00|   12|
|9/6/2014 11:59:00|   19|
+-----------------+-----+
only showing top 20 rows



In [155]:
df_2.show()

+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|4/1/2014 0:11:00| 40.769|-73.9549|B02512|
|4/1/2014 0:17:00|40.7267|-74.0345|B02512|
|4/1/2014 0:21:00|40.7316|-73.9873|B02512|
|4/1/2014 0:28:00|40.7588|-73.9776|B02512|
|4/1/2014 0:33:00|40.7594|-73.9722|B02512|
|4/1/2014 0:33:00|40.7383|-74.0403|B02512|
|4/1/2014 0:39:00|40.7223|-73.9887|B02512|
|4/1/2014 0:45:00| 40.762| -73.979|B02512|
|4/1/2014 0:55:00|40.7524| -73.996|B02512|
|4/1/2014 1:01:00|40.7575|-73.9846|B02512|
|4/1/2014 1:19:00|40.7256|-73.9869|B02512|
|4/1/2014 1:48:00|40.7591|-73.9684|B02512|
|4/1/2014 1:49:00|40.7271|-73.9803|B02512|
|4/1/2014 2:11:00|40.6463|-73.7896|B02512|
|4/1/2014 2:25:00|40.7564|-73.9167|B02512|
|4/1/2014 2:31:00|40.7666|-73.9531|B02512|
|4/1/2014 2:43:00| 40.758|-73.9761|B02512|
|4/1/2014 3:22:00|40.7238|-73.9821|B02512|
|4/1/2014 3:35:00|40.7531|-74.0039|B02512|
|4/1/2014 3:35:00|40.7389|-74.0393|B02512|
+----------

In [190]:
def fix_all_trip_data_timestamp(timestamp_):
	return datetime.strptime(timestamp_.split(' ')[0], '%d/%m/%Y').strftime('%Y-%m-%d')


def fix_all_trip_data_timestamp2(timestamp_):
	return datetime.strptime(timestamp_ ,'%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d')



In [191]:
spark.udf.register("fixtimestamp", fix_all_trip_data_timestamp)


spark.udf.register("fixtimestamp2", fix_all_trip_data_timestamp2)




<function __main__.fix_all_trip_data_timestamp2>

In [189]:
datetime.strptime(df_2.select('Date/Time').take(4)[0]['Date/Time'] ,'%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d')



'2014-01-04'

In [ ]:
datetime.strptime(df_2.select('Date/Time').take(4)[0]['Date/Time'].split(' ')[0],'%d/%m/%Y').strftime('%Y-%m-%d')



In [183]:
df_2.select('Date/Time').take(4)[0]['Date/Time']

'4/1/2014 0:11:00'

In [172]:
df_2.show(3)

+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|4/1/2014 0:11:00| 40.769|-73.9549|B02512|
|4/1/2014 0:17:00|40.7267|-74.0345|B02512|
|4/1/2014 0:21:00|40.7316|-73.9873|B02512|
+----------------+-------+--------+------+
only showing top 3 rows



In [204]:
spark_RDD.take(3)

[Row(Date/Time='4/1/2014 0:11:00', Lat='40.769', Lon='-73.9549', Base='B02512'),
 Row(Date/Time='4/1/2014 0:17:00', Lat='40.7267', Lon='-74.0345', Base='B02512'),
 Row(Date/Time='4/1/2014 0:21:00', Lat='40.7316', Lon='-73.9873', Base='B02512')]

In [215]:
transformed_RDD = spark_RDD.map(
          lambda x: Row(
          Lat=x['Lat'],
          Lon= x['Lon'],
          Base=x['Base'],
          timestamp= datetime.strptime(x['Date/Time'], '%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d') ))\
          .take(30)

In [217]:
transformed_RDD = spark_RDD.map(
          lambda x: Row(
          Lat=x['Lat'],
          Lon= x['Lon'],
          Base=x['Base'],
          timestamp= datetime.strptime(x['Date/Time'], '%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d') ))


In [223]:
transformed_RDD = spark_RDD.map(
                lambda x: Row(
                Lat=x['Lat'],
                Lon= x['Lon'],
                Base=x['Base'],
                timestamp= datetime.strptime(x['Date/Time'], '%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d') ))\
                .map(lambda x : (x.timestamp, 1))\
                .reduceByKey(add)\
                .sortBy(lambda x : x[1], False)\
                .take(30)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 239.0 failed 1 times, most recent failure: Lost task 3.0 in stage 239.0 (TID 1678, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yennanliu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/Users/yennanliu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/yennanliu/anaconda3/envs/pyspark_/lib/python3.5/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/yennanliu/anaconda3/envs/pyspark_/lib/python3.5/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/y

In [218]:
transformed_RDD.take(10)

[Row(Base='B02512', Lat='40.769', Lon='-73.9549', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7267', Lon='-74.0345', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7316', Lon='-73.9873', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7588', Lon='-73.9776', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7594', Lon='-73.9722', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7383', Lon='-74.0403', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7223', Lon='-73.9887', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.762', Lon='-73.979', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7524', Lon='-73.996', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7575', Lon='-73.9846', timestamp='2014-01-04')]

In [214]:
date_count = transformed_RDD\
            .map(lambda x : (x['Lon'], 1))\
            .reduceByKey(add)\
            .sortBy(lambda x : x[1], False)\
            .take(30)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 231.0 failed 1 times, most recent failure: Lost task 3.0 in stage 231.0 (TID 1664, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yennanliu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/Users/yennanliu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/yennanliu/anaconda3/envs/pyspark_/lib/python3.5/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/yennanliu/anaconda3/envs/pyspark_/lib/python3.5/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File 

In [200]:
spark_RDD = df_2.rdd
digested_RDD = spark_RDD.map(
              lambda x: Row(
              timestamp= datetime.strptime(x['Date/Time'], '%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d') ))\
              .take(30)

In [201]:
digested_RDD

[Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04'),
 Row(timestamp='2014-01-04')]

In [167]:
df_2.select('Date/Time').take(3)

[Row(Date/Time='4/1/2014 0:11:00'),
 Row(Date/Time='4/1/2014 0:17:00'),
 Row(Date/Time='4/1/2014 0:21:00')]

In [126]:
df_.show()

+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|
|9/1/2014 0:01:00|  40.75|-74.0027|B02512|
|9/1/2014 0:03:00|40.7559|-73.9864|B02512|
|9/1/2014 0:06:00| 40.745|-73.9889|B02512|
|9/1/2014 0:11:00|40.8145|-73.9444|B02512|
|9/1/2014 0:12:00|40.6735|-73.9918|B02512|
|9/1/2014 0:15:00|40.7471|-73.6472|B02512|
|9/1/2014 0:16:00|40.6613|-74.2691|B02512|
|9/1/2014 0:32:00|40.3745|-73.9999|B02512|
|9/1/2014 0:33:00|40.7633|-73.9773|B02512|
|9/1/2014 0:33:00|40.7467|-73.6131|B02512|
|9/1/2014 0:37:00|40.8105|  -73.96|B02512|
|9/1/2014 0:38:00| 40.679|-74.0111|B02512|
|9/1/2014 0:39:00|40.4023|-73.9839|B02512|
|9/1/2014 0:48:00|40.7378|-74.0395|B02512|
|9/1/2014 0:48:00|40.7214|-73.9884|B02512|
|9/1/2014 0:49:00|40.8646|-73.9081|B02512|
|9/1/2014 1:08:00|40.7398|-74.0061|B02512|
|9/1/2014 1:17:00|40.6793|-74.0116|B02512|
|9/1/2014 1:19:00|40.7328|-73.9875|B02512|
+----------

In [144]:
mytime= '9/1/2014 0:01:00'
mytime.split(' ')[0]

'9/1/2014'

In [154]:
from datetime import datetime
datetime.strptime(mytime.split(' ')[0], '%d/%m/%Y').strftime('%Y-%m-%d')

'2014-01-09'

In [146]:
mytime.split(' ')[0].strftime('%Y-%m-%d')

AttributeError: 'str' object has no attribute 'strftime'

In [145]:
type(mytime.split(' ')[0])

str

In [224]:
df_2.show(3)

+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|4/1/2014 0:11:00| 40.769|-73.9549|B02512|
|4/1/2014 0:17:00|40.7267|-74.0345|B02512|
|4/1/2014 0:21:00|40.7316|-73.9873|B02512|
+----------------+-------+--------+------+
only showing top 3 rows



In [264]:
spark_RDD = df_2.rdd
transformed_RDD = spark_RDD.map(
                lambda x: Row(
                Lat=x['Lat'],
                Lon= x['Lon'],
                Base=x['Base'],
                timestamp= datetime.strptime(x['Date/Time'], '%m/%d/%Y %H:%M:%S').strftime('%Y-%m-%d') ))\
                .collect()
                

In [268]:
len(transformed_RDD)

564516

In [269]:
transformed_RDD

[Row(Base='B02512', Lat='40.769', Lon='-73.9549', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7267', Lon='-74.0345', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7316', Lon='-73.9873', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7588', Lon='-73.9776', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7594', Lon='-73.9722', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7383', Lon='-74.0403', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7223', Lon='-73.9887', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.762', Lon='-73.979', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7524', Lon='-73.996', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7575', Lon='-73.9846', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7256', Lon='-73.9869', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7591', Lon='-73.9684', timestamp='2014-04-01'),
 Row(Base='B02512', Lat='40.7271', Lon='-73.9803', timestamp='2014-04-01'),
 Row(Base='B0251

In [270]:
sc.parallelize(transformed_RDD)

ParallelCollectionRDD[542] at parallelize at PythonRDD.scala:175

In [258]:
sc.parallelize(transformed_RDD).take(3)

[Row(Base='B02512', Lat='40.769', Lon='-73.9549', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7267', Lon='-74.0345', timestamp='2014-01-04'),
 Row(Base='B02512', Lat='40.7316', Lon='-73.9873', timestamp='2014-01-04')]

In [ ]:
sc.parallelize(transformed_RDD).take(3)

In [271]:
sc.parallelize(transformed_RDD).map(lambda x : (x.timestamp, 1))\
               .reduceByKey(add)\
               .take(30)\
    

[('2014-04-16', 17717),
 ('2014-04-27', 14677),
 ('2014-04-30', 36251),
 ('2014-04-17', 20973),
 ('2014-04-09', 16843),
 ('2014-04-06', 13445),
 ('2014-04-28', 15475),
 ('2014-04-11', 20420),
 ('2014-04-14', 12674),
 ('2014-04-19', 14602),
 ('2014-04-26', 24925),
 ('2014-04-08', 16188),
 ('2014-04-15', 20641),
 ('2014-04-24', 23352),
 ('2014-04-21', 13162),
 ('2014-04-07', 19550),
 ('2014-04-20', 11017),
 ('2014-04-23', 20346),
 ('2014-04-02', 17474),
 ('2014-04-01', 14546),
 ('2014-04-25', 25095),
 ('2014-04-12', 18170),
 ('2014-04-03', 20701),
 ('2014-04-22', 16975),
 ('2014-04-29', 22835),
 ('2014-04-10', 20041),
 ('2014-04-18', 18074),
 ('2014-04-05', 19521),
 ('2014-04-13', 12112),
 ('2014-04-04', 26714)]

In [272]:
spark_RDD = df_2.rdd
# dataframe -> RDD 
date_lon_lat_list = spark_RDD.map(
                lambda x: Row(
                Lat=x['Lat'],
                Lon= x['Lon'],
                Base=x['Base'],
                timestamp= datetime.strptime(x['Date/Time'], '%m/%d/%Y %H:%M:%S').strftime('%Y-%m-%d') ))\
                .collect()
# list -> RDD
transformed_RDD_updated_RDD =  sc.parallelize(date_lon_lat_list)
date_count = transformed_RDD_updated_RDD.map(lambda x : (x.timestamp, 1))\
                .reduceByKey(add)\
                .sortBy(lambda x : x[1], False)\
                .take(30)

In [273]:
date_count

[('2014-04-30', 36251),
 ('2014-04-04', 26714),
 ('2014-04-25', 25095),
 ('2014-04-26', 24925),
 ('2014-04-24', 23352),
 ('2014-04-29', 22835),
 ('2014-04-17', 20973),
 ('2014-04-03', 20701),
 ('2014-04-15', 20641),
 ('2014-04-11', 20420),
 ('2014-04-23', 20346),
 ('2014-04-10', 20041),
 ('2014-04-07', 19550),
 ('2014-04-05', 19521),
 ('2014-04-12', 18170),
 ('2014-04-18', 18074),
 ('2014-04-16', 17717),
 ('2014-04-02', 17474),
 ('2014-04-22', 16975),
 ('2014-04-09', 16843),
 ('2014-04-08', 16188),
 ('2014-04-28', 15475),
 ('2014-04-27', 14677),
 ('2014-04-19', 14602),
 ('2014-04-01', 14546),
 ('2014-04-06', 13445),
 ('2014-04-21', 13162),
 ('2014-04-14', 12674),
 ('2014-04-13', 12112),
 ('2014-04-20', 11017)]